# Create MVImgNet bins

This notebook creates the bins for the MVImgNet dataset.

### Running for different categories 

Only need to change CLASS_NUM variable.

All categories we are using in mvimgnet:
`100  113  125  126  152  166  19  196  23  46  57  60  7  70  8  99`

So far we have done the following:
`7`,`8`, `70`, `100`, `19`, `46`, || `57`, `60`, `99`, `113`, `125`, `126`, `152`, `166`, `196`

Doesn't work for class `23` as it's biggest max angle = 84.257

In [220]:
CLASS_NUM = 152

In [120]:
CONFIG = {"input_path": "/home/scur0542/open-hummingbird-eval/datasets/mvimgnet/",
    "output_path": "/home/scur0542/open-hummingbird-eval/datasets/split_angles_mvimagenet/",
    "colmap_script": "/home/scur0542/open-hummingbird-eval/colmap/scripts/python",
    "ANGLE_BINS": [0, 15, 30, 45, 60, 75, 90],
    "class_mapping": {23: "Laptop", 46: "Bed", 19: "Microwave", 99: "Coat Rack", 126: "Sink", 125: "Toilet", 
    8: "Sofa", 166: "Broccoli", 7: "Stove", 152: "Strings", 196: "Durian", 113: "Ceiling Lamp", 70: "Toy Dragon",
    60: "Toy Cow", 100: "Guitar Stand", 57: "Toy Cat"}
}

### Imports

In [2]:
import os
import csv
import json
import shutil
import cv2
import numpy as np
from tqdm import tqdm
import math
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
from datetime import datetime

In [4]:
# TEST function


tot_sum = 0
for j in [7, 8, 19, 46, 57, 60, 70, 99, 100, 113, 125, 126, 152, 166, 196]:
#  60, 113, 152
    for i in [0, 15, 30, 45, 60, 75, 90]:
        dirs_mask = os.listdir(f'/home/scur0542/open-hummingbird-eval/datasets/split_angles_mvimagenet/{j}/{i}/mask')
        dirs_img = os.listdir(f'/home/scur0542/open-hummingbird-eval/datasets/split_angles_mvimagenet/{j}/{i}/img')

        print(f'len {i} : {len(dirs_mask), len(dirs_img)}'  )

        tot = 0
        for img, mask in zip(sorted(dirs_img), sorted(dirs_mask)):
            
            mask = mask[:-4]
            if img == mask:
                # print('corr ',mask, img)
                tot += 1
            else:
                print(mask, img)
        
        print(f'correct : {tot}')
    tot_sum += tot
print(tot_sum)

len 0 : (197, 197)
correct : 197
len 15 : (197, 197)
correct : 197
len 30 : (197, 197)
correct : 197
len 45 : (197, 197)
correct : 197
len 60 : (197, 197)
correct : 197
len 75 : (197, 197)
correct : 197
len 90 : (197, 197)
correct : 197
len 0 : (91, 91)
correct : 91
len 15 : (91, 91)
correct : 91
len 30 : (91, 91)
correct : 91
len 45 : (91, 91)
correct : 91
len 60 : (91, 91)
correct : 91
len 75 : (91, 91)
correct : 91
len 90 : (91, 91)
correct : 91
len 0 : (120, 120)
correct : 120
len 15 : (120, 120)
correct : 120
len 30 : (120, 120)
correct : 120
len 45 : (120, 120)
correct : 120
len 60 : (120, 120)
correct : 120
len 75 : (120, 120)
correct : 120
len 90 : (120, 120)
correct : 120
len 0 : (23, 23)
correct : 23
len 15 : (23, 23)
correct : 23
len 30 : (23, 23)
correct : 23
len 45 : (23, 23)
correct : 23
len 60 : (23, 23)
correct : 23
len 75 : (23, 23)
correct : 23
len 90 : (23, 23)
correct : 23
len 0 : (762, 762)
correct : 762
len 15 : (762, 762)
correct : 762
len 30 : (762, 762)
correct

In [31]:
# from pathlib import Path
# path  = Path('datasets/split_angles_mvimagenet/57/0/img/2a01190a_001.jpg')

# parent_folder = path.parent.parent.parent.name 
# # /home/scur0542/open-hummingbird-eval/datasets/split_angles_mvimagenet/70/0/img/0000a9b5_001.jpg
# print(parent_folder == '70')
# print(parent_folder)

### Obtain the read_write_model from python
In the open-hummingbird-eval, go to colmap, there is a scripts folder with a python which contains read_write_model.py 

The following functions are needed: 
read_images_binary & qvec2rotmat

In [221]:
sys.path.append(CONFIG["colmap_script"])

from read_write_model import read_images_binary, qvec2rotmat

In [222]:
def setup_paths(CLASS_NUM, input_base, output_base):
    image_path = input_base + 'images/'
    mask_path = input_base + 'mask/'
    class_image_folder = image_path + str(CLASS_NUM) + '/'
    class_mask_folder = mask_path + str(CLASS_NUM) + '/'
    output_folder = output_base + str(CLASS_NUM) + '/'
    os.makedirs(output_folder, exist_ok=True)
    return class_image_folder, class_mask_folder, output_folder

# Creating per class the (img & mask) inputs path and the output folder
class_image_folder_path, class_mask_folder_path, output_folder = setup_paths(CLASS_NUM, CONFIG["input_path"], CONFIG["output_path"])

In [223]:
# print(class_image_folder_path, class_mask_folder_path, output_folder)

In [224]:
def create_outputs_path(class_number, output_path, angle_bin):
    # Naming files that go in the output
    TXT_PATH = output_path + str(class_number) + '.txt'
    CSV_PATH = output_path + str(class_number) + '.csv'
    JSON_PATH = output_path + str(class_number) + '.json'
    # Create the angle folders (0-90) in the output folder
    for i in angle_bin:
        os.makedirs(output_path+str(i), exist_ok=True)
        os.makedirs(output_path+str(i)+'/img', exist_ok=True)
        os.makedirs(output_path+str(i)+'/mask', exist_ok=True)
    return TXT_PATH, CSV_PATH, JSON_PATH

TXT_NAME, CSV_NAME, JSON_NAME = create_outputs_path(CLASS_NUM, output_folder, CONFIG["ANGLE_BINS"])

In [225]:
# print(TXT_NAME, CSV_NAME, JSON_NAME)

In [226]:
def compute_angles(images):
    sorted_images = sorted(images.items(), key=lambda item: int(item[1].name.split(".")[0]))
    angles_list = [0]
    for i in range(1, len(sorted_images)):
        prev = sorted_images[0][0]
        curr = sorted_images[i][0]

        image1 = images[prev]
        image2 = images[curr]

        R1 = qvec2rotmat(image1.qvec)
        R2 = qvec2rotmat(image2.qvec)

        R_rel = R2 @ R1.T

        angle_rad = np.arccos((np.trace(R_rel) - 1) / 2)
        angle_deg = np.degrees(angle_rad)

        angles_list.append(round(angle_deg, 3))
    return angles_list

In [227]:
def compute_steps(angles_list):
    steps_list = []
    for i in range(1, len(angles_list)):

        prev_angle = angles_list[i-1]
        curr_angle = angles_list[i]
        steps_list.append(round(abs(prev_angle-curr_angle), 3))
    return steps_list

In [228]:
def compute_and_log_angles(path_class_img, csv_path, angle_func = compute_angles, step_func = compute_steps):
    """
    Compute angles and steps from COLMAP extrinsics for each image folder,
    write results to CSV, and return nested dictionary with computed values.

    Args:
        path_class_img (str): Path to class image folders.
        csv_path (str): Output CSV path.
        angle_func (function): Function to compute angles (e.g., compute_angles).
        step_func (function): Function to compute steps (e.g., compute_steps).

    Returns:
        nested_dic (dict): nested dictionary with angles and steps per image folder.
    """
    # Initialize nested dictionary
    nested_dict = {}

    # Write CSV headers
    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Create the Column Headers for the CSV file
        # NOTE: 3D_points is still unused but kept for compatibility
        writer.writerow(['id_img', 'range', 'min_angle', 'min_idx', 'max_angle', 'max_idx', 'num_img', 'step_mean', 'step_std', 'min_step', 'max_step', '3D_points'])

    # Calculate angles with Colmap extrinsics for each image subsequently
    sorted_folder = sorted(os.listdir(path_class_img))
    print("Total files : ", len(sorted_folder))

    # Loop over image folders
    for filename in tqdm(sorted_folder):
        file_cl_image_folder_path = path_class_img + filename
        folder_path = file_cl_image_folder_path + '/images/'

        # Get image files
        if not os.path.exists(folder_path):
            # skip if no images folder
            print("There is not a folder for images")
            continue

        list_images = []
        for f in os.listdir(folder_path):
            image_name = folder_path + f
            list_images.append(image_name)
        x = np.sort(list_images)

        if not list_images:
            # skip if no images found
            print("There are no images in this folder")
            continue

        # Read COLMAP binary
        bin2 = file_cl_image_folder_path +'/sparse/0/images.bin'
        if not os.path.exists(bin2):
            # skip sparse folder that don't have colmap stats
            print("The sparse folder does not contain COLMAP statistics")
            continue
        
        images = read_images_binary(bin2)

        # Compute angles & steps
        angle_list = angle_func(images)
        step_list = step_func(angle_list)

        # Collect stats
        min_angle = min(angle_list)
        max_angle = max(angle_list)
        idx_min_angle = np.argmin(angle_list)
        idx_max_angle = np.argmax(angle_list)

        dic = {
            'angles': angle_list,
            'step': step_list
        }

        nested_dict[filename] = dic

        # Write per row in csv.file
        with open(csv_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            step_mean = float(np.round(np.mean(step_list), 2))
            step_std = float(np.round(np.std(step_list), 2))
            min_step = float(np.round(np.min(step_list), 2))
            max_step = float(np.round(np.max(step_list), 2))

            writer.writerow([filename,
                             abs(round(min_angle, 3) - round(max_angle, 3)),
                             round(min_angle, 3), idx_min_angle,
                             round(max_angle, 3), idx_max_angle,
                             len(list_images),
                             step_mean, step_std,
                             min_step, max_step])  # Placeholder for 3D_points column if needed

    return nested_dict


nested_dic = compute_and_log_angles(class_image_folder_path, CSV_NAME)

Total files :  225


100%|██████████| 225/225 [00:21<00:00, 10.67it/s]


### What is the stucture of the JSON file?

```
{
  "some_id": {
    "angles": [...],
    "step": [...]
  },
  "another_id": {
    "angles": [...],
    "step": [...]
  },
  ...
}
```

1. Keys (like "00000b90", "00001d21", etc.):
    1. These are identifiers (likely strings from filenames, image IDs, sample IDs, etc.).
    2. Every ID corresponds to one object/entity.
2. Values (dictionaries with "angles" and "step"):
    1. Each ID maps to a dictionary with:
      1. "angles": A list of floats for the computed rotation angles of each image in that folder, relative to the first image (in degrees).
      2. "step": A list of floats for the differences between consecutive angles (how much the view changed between each image).


#### Example Entry:
```
"00000b90": {
  "angles": [0, 3.831, 9.897, 15.838, ...],
  "step": [3.831, 6.066, 5.941, 6.274, ...]
},
  "00001d21": {
    "angles": [
      0,
      0.374,
      1.748,
      4.001, ...
    ],
    "step": [
      0.374,
      1.374,
      2.253,
      3.397, ...
    ]
  }
```

In [229]:
# Write the nested dictionary to a binary file
with open(JSON_NAME, 'w') as file:
    json.dump(nested_dic, file, indent=2)

In [230]:
def filter_ids_by_angle(csv_name, txt_name, final_angle):
    """
    Filters IDs from a CSV file where 'max_angle' >= final_angle
    and appends them to a text file (one per line).

    Parameters:
    - csv_name (str): Path to the CSV file.
    - txt_name (str): Path to the output TXT file.
    - final_angle (float): Threshold to filter 'max_angle'.
    """

    # Filter angle range, then add the img_id to a txt file
    df = pd.read_csv(csv_name)
    ids_to_write = df[df['max_angle'] >= final_angle]['id_img']
    
    if ids_to_write.empty:
        print(f'The biggest angle (= {max(df["max_angle"])}) of this category ({CONFIG["class_mapping"][CLASS_NUM]}) is smaller than the given final angle (= {final_angle})')
        raise ValueError(f'No images found with angle greater or equal to {final_angle}. TXT file for {CONFIG["class_mapping"][CLASS_NUM]} will be empty. TXT file not created.')
        return

    with open(txt_name, 'a') as f:
        for idd in ids_to_write:
            f.write(f"{idd}\n")

FINAL_ANGLE = CONFIG["ANGLE_BINS"][-1]
filter_ids_by_angle(CSV_NAME, TXT_NAME, FINAL_ANGLE)

In [231]:
def process_and_copy_images(class_image_folder_path, class_mask_folder_path, output_folder, json_name = JSON_NAME, txt_name = TXT_NAME, angle_bins = CONFIG["ANGLE_BINS"]):
    """
    Processes images and masks based on angle bins, copies them to new folders, 
    and computes error statistics.

    Parameters:
    - json_name (str): Path to the JSON file with angle information.
    - txt_name (str): Path to the TXT file with selected image IDs.
    - class_image_folder_path (str): Path to the image folders.
    - class_mask_folder_path (str): Path to the mask folders.
    - output_folder (str): Destination folder for processed images and masks.
    - angle_bins (list): A list containing the desired bins for the angles (from CONFIG["ANGLE_BINS"])

    Returns:
    - (float, float): Standard deviation and mean of angle errors.
    """

    # Load the nested dictionary from the previously created JSON file
    with open(json_name, 'r') as f:
        nested_dict = json.load(f)


    if not os.path.exists(txt_name):
        raise ValueError(f"Warning: {txt_name} does not exist. Skipping processing. Please delete this category.")
        return

    # Load names of image folders that have enough degrees
    with open(txt_name, 'r') as f:
        # Returns a list of strings
        data = [line.strip() for line in f]
    
    if not data:
        raise ValueError("Warning: No images found to process. TXT file is empty.")
        return

    # Add the images and mask in the folders
    tot_err = []

    for img_folder in tqdm(data):
        source_folder = class_image_folder_path + str(img_folder) +'/images/'
        source_folder_2 = class_mask_folder_path + str(img_folder)

        images = [f for f in sorted(os.listdir(source_folder)) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        images_2 = [f for f in sorted(os.listdir(source_folder_2)) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

        # skip when images or masks are missing in either lists
        if len(images_2) != len(images):
            continue

        angles = nested_dict[img_folder]['angles']

        i_want_angles = []
        i_want_masks = []

        for angle_bin in angle_bins:
            closest = min(angles, key=lambda x: abs(x - angle_bin))
            index = angles.index(closest)
            tot_err.append(closest - angle_bin)

            i_want_angles.append(images[index])
            i_want_masks.append(images_2[index])

        # Copy images to the corresponding folders
        for i, img in enumerate(i_want_angles):
            folder_img = output_folder + str(angle_bins[i]) + '/img'

            src_path = os.path.join(source_folder, img)
            dst_path = os.path.join(folder_img, f"{img_folder}_{img}")
            shutil.copy(src_path, dst_path)

        # Copy masks to the corresponding folders
        for i, img in enumerate(i_want_masks):
            folder_mask = output_folder + str(angle_bins[i]) + '/mask'
            src_path = os.path.join(source_folder_2, img)
            dst_path = os.path.join(folder_mask, f"{img_folder}_{img}")
            shutil.copy(src_path, dst_path)

    # IF ERROR std is smaller than 5 degrees

    # Calculate the mean and standard deviation of the error
    std_err = round(np.std(tot_err), 2)
    mean_err = round(np.mean(tot_err), 2)

    print(f"std of error: {std_err}\nmean of error: {mean_err}")

    return data, std_err, mean_err

In [232]:
with open('/home/scur0542/open-hummingbird-eval/datasets/split_angles_mvimagenet/70/70.txt', 'r') as f:
    # Returns a list of strings
    data = [line.strip() for line in f]

len(data)

627

In [233]:
data, std, mean = process_and_copy_images(class_image_folder_path, class_mask_folder_path, output_folder)

100%|██████████| 192/192 [00:06<00:00, 31.70it/s]

std of error: 1.24
mean of error: 0.04


In [234]:
def log_errors(mean_error, std_error, num_images_processed, class_num, class_cat, output_folder, csv_folder, global_log_csv='processing_log.csv'):
    # Global CSV log entry
    log_entry = {
        'run_timestamp': datetime.now().strftime("%Y-%m-%d %H:%M"),
        'class_num': class_num,
        'class_category' : class_cat,
        'std_error': std_error,
        'mean_error': mean_error,
        'num_images_processed': num_images_processed
    }

    log_df = pd.DataFrame([log_entry])

    # Append to global log CSV
    path_log_csv = os.path.join(csv_folder, global_log_csv)
    if os.path.exists(path_log_csv):
        log_df.to_csv(path_log_csv, mode='a', header=False, index=False)
    else:
        log_df.to_csv(path_log_csv, index=False)

    # Local stats summary per class
    summary_file = os.path.join(output_folder, 'stats_summary.txt')
    with open(summary_file, 'w') as f:
        f.write(f"Class: {class_num}\n")
        f.write(f"Mean error: {mean_error:.2f} degrees\n")
        f.write(f"Std error: {std_error:.2f} degrees\n")
        f.write(f"num_images_processed: {num_images_processed} img\n")



    print(f'Logged errors for class {class_num} to global CSV and summary file.')

# Usage example:
log_errors(mean, std, len(data), CLASS_NUM, CONFIG["class_mapping"][CLASS_NUM], output_folder, CONFIG["output_path"])


Logged errors for class 152 to global CSV and summary file.


In [235]:
def plot_class_angle_histogram(class_number, histogram_bins = 100):
    output_path = CONFIG["output_path"]
    class_mapping = CONFIG["class_mapping"]

    if class_number not in class_mapping:
        raise ValueError(f"Class number {class_number} is not in the class mapping.")

    category = class_mapping[class_number]
    output_folder = output_path + str(class_number) + '/'
    csv_name = output_folder + str(class_number) + '.csv'

    if not os.path.exists(csv_name):
        raise FileNotFoundError(f"CSV file not found: {csv_name}")

    df = pd.read_csv(csv_name)
    print(f"Loaded CSV with {df.shape[0]} rows.")

    if 'max_angle' not in df.columns:
        raise ValueError(f"'max_angle' column not found in {csv_name}")

    # Optional: Downsample if too large (e.g., > 100,000 rows)
    if df.shape[0] > 100000:
        df = df.sample(100000, random_state=42)
        print("Data was downsampled to 100,000 rows for performance.")

    sns.histplot(df['max_angle'], bins= histogram_bins, kde=True, color='green', edgecolor='gray')

    plt.xlabel('Angle in degrees')
    plt.ylabel(f'Instances of class {class_number}: \"{category}\"')
    plt.axvline(x=90, color='red', linestyle='--', label='90 degrees')
    plt.legend()
    plt.title(f'Class {class_number} \"{category}\": max angle histogram')
    plt.tight_layout()

    # Save plot
    plot_filename = os.path.join(output_folder, f"{class_number}_angle_histogram.png")
    plt.savefig(plot_filename)
    print(f"Plot saved as: {plot_filename}")
    plt.close()

    return plot_filename

In [236]:
# Create the histogram for the CLASS explored
histogram_path = plot_class_angle_histogram(CLASS_NUM)
# display(Image(filename=histogram_path))

Loaded CSV with 225 rows.
Plot saved as: /home/scur0542/open-hummingbird-eval/datasets/split_angles_mvimagenet/152/152_angle_histogram.png
